### --> PIPELINE QUI CONSTRUIT F1 et F2

### IMPORT

In [1]:
import datetime
import urllib.request
import csv
import os
import subprocess
import tweepy
import pandas as pd
import json
from lxml import html
import requests
import numpy as np
import math
from tensorflow.python.lib.io import file_io
#jhvkfvhkqhndvv

### DEFINITION FONCTIONS

In [2]:
def nom_normalizer(equipe_a_normaliser):
    teams = {
        'OLYMPIQUE LYONNAIS': ['OLYMPIQUE LYONNAIS', 'LYON', 'OL', 'O.L.'], 
        'AC AJACCIO': ['AC AJACCIO', 'AJACCIO', 'ACA', 'A.C.A', 'ATHLETIC CLUB AJACCIEN','A-C AJACCIO','A.C AJACCIO'],
        'RC STRASBOURG ALSACE' : ['STRASBOURG','RACING CLUB DE STRASBOURG ALSACE', 'RC STRASBOURG','R.C. STRASBOURG','R.C.S.','RCS', 'R.C.S.A.', 'RC STRASBOURG ALSACE', 'R.C. STRASBOURG ALSACE'], 
        'AS MONACO': ['AS MONACO', 'MONACO', 'ASSOCIATION SPORTIVE DE MONACO FOOTBALL CLUB', 'AS MONACO FC', 'A.S MONACO F.C', 'ASM'], 
        'CLERMONT FOOT 63': ['CLERMONT FOOT 63', 'CLERMONT-FERRAND', 'CF63', 'C.F.63', 'C-F 63', 'CLERMONT FOOT'], 
        'PARIS SAINT GERMAIN': ['PARIS SAINT GERMAIN', 'PARIS', 'PARIS-SAINT-GERMAIN FOOTBALL CLUB', 'PSG', 'P.S.G.', 'PARIS SG', 'PARIS-SG', 'PARIS SAINT-GERMAIN', 'PARIS SAINT-GERMAIN FOOTBALL CLUB', 'PARIS SAINT-GERMAIN FC'], 
        'TOULOUSE FC' : ['TOULOUSE', 'TOULOUSE FOOTBALL CLUB', 'TFC', 'T.F.C.', 'TOULOUSE FC', 'TOULOUSE F.C.'], 
        'OGC NICE': ['NICE', "OLYMPIQUE GYMNASTE CLUB NICE COTE D'AZUR", 'OGC NICE', 'O.G.C. NICE', "OGC NICE COTE D'AZUR", "O.G.C. NICE COTE D'AZUR", 'OLYMPIQUE GYMNASTE CLUB NICE'], 
        'ANGERS SCO': ['ANGERS SCO', 'ANGERS', "SCO D'ANGERS", 'SCOA', "ANGERS SPORTING CLUB DE L'OUEST", 'SCO ANGERS', 'ANGERS-SCO', 'ANGERS S.C.O.', 'S.C.O. ANGERS'], 
        'FC NANTES' : ['FC NANTES', 'NANTES', 'FOOTBALL CLUB DE NANTES', 'F.C. NANTES', 'FCN', 'F.C.N.'], 
        'RC LENS': ['RC LENS', 'LENS', 'RACING CLUB DE LENS', 'RCL', 'R.C.L.', 'R.C. LENS'], 
        'STADE BRESTOIS': ['STADE BRESTOIS', 'BREST', 'STADE BRESTOIS 29', 'SB29', 'S.B.29', 'S.B. 29'], 
        'LOSC LILLE': ['LOSC LILLE', 'LILLE', 'LOSC', 'LILLE OLYMPIQUE SPORTING CLUB', 'L.O.S.C.', 'L.O.S.C. LILLE'], 
        'AJ AUXERRE': ['AJ AUXERRE', 'AUXERRE', 'AJA', 'ASSOCIATION DE LA JEUNESSE AUXERROISE', 'A.J.AUXERRE', 'A.J AUXERRE'], 
        'MONTPELLIER HÉRAULT SC': ['MONTPELLIER HÉRAULT SC','MONTPELLIER HÉRAULT', 'MONTPELLIER', 'MONTPELLIER-HERAULT SPORT CLUB', 'MONTPELLIER HERAULT SPORT CLUB', 'MONTPELLIER HSC', 'MHSC', 'MONTPELLIER-HERAULT SC', 'MONTPELLIER-HÉRAULT SC', 'MONTPELLIER-HERAULT S.C.', 'M.H.S.C.', 'MONTPELLIER HERAULT SC', 'MONTPELLIER HÉRAULT SC', 'MONTPELLIER HERAULT S.C.'], 
        'ESTAC TROYES': ['TROYES', 'ESPERANCE SPORTIVE TROYES AUBE CHAMPAGNE', 'ESTAC', 'E.S.T.A.C.', 'ESTAC TROYES', 'E.S.T.A.C. TROYES'], 
        'STADE RENNAIS FC': ['RENNES', 'STADE RENNAIS FOOTBALL CLUB', 'STADE RENNAIS', 'STADE RENNAIS FC', 'S.R.F.C.', 'STADE RENNAIS F.C.'], 
        'FC LORIENT': ['FC LORIENT', 'LORIENT', 'FOOTBALL CLUB LORIENT BRETAGNE SUD', 'FCL', 'F.C.L.', 'F.C. LORIENT', 'FC LORIENT BRETAGNE SUD', 'FOOTBALL CLUB LORIENTAIS'], 
        'OLYMPIQUE DE MARSEILLE': ['OLYMPIQUE DE MARSEILLE', 'MARSEILLE', 'OM', 'O.M.'],
        'STADE DE REIMS': ['STADE DE REIMS', 'REIMS', 'STADE REIMS'],
        'DIJON': ['DIJON', 'DIJON FCO', 'DFCO', "DIJON FOOTBALL COTE-D'OR", "DIJON FOOTBALL CÔTE-D'OR"],
        'SAINT ETIENNE': ['ST ETIENNE', 'ST-ETIENNE', 'ST ÉTIENNE', 'ST-ÉTIENNE', 'SAINT ETIENNE', 'SAINT-ETIENNE', 'SAINT ÉTIENNE', 'SAINT-ÉTIENNE', 'ASSE','AS SAINT ETIENNE', 'AS SAINT-ETIENNE', 'AS SAINT-ÉTIENNE', 'AS SAINT-ÉTIENNE'],
        'BORDEAUX': ['BORDEAUX', 'FOOTBALL CLUB DES GIRONDINS DE BORDEAUX', 'FC GIRONDINS DE BORDEAUX', 'GIRONDINS DE BORDEAUX', 'FCG BORDEAUX', 'FOOTBALL CLUB GIRONDINS DE BORDEAUX'],
        'METZ': ['METZ', 'FC METZ', 'FOOTBALL CLUB DE METZ', 'FOOTBALL CLUB METZ']}
    resultat = equipe_a_normaliser
    for equipe_name in teams:
        if equipe_a_normaliser in teams[equipe_name]:
            return equipe_name
    return resultat

def merge_season(season, path):
    directory = f"./Data_final/"
    if not os.path.exists(directory):
        os.mkdir(directory)
    merged =open(directory+f"season_{season}_{season+1}.csv","a")    
    merged.write('Journee,HomeTeam,AwayTeam,FTHG,FTAG,URL\n')  
    for num in range(1,39):
        f = open(path+"/season_"+str(season)+"_"+str(season+1)+"_journee_"+str(num)+".csv")
        next(f) # skip the header
        for line in f:
            merged.write(f'{num},'+line)
        f.close() # not really needed
    merged.close()

def merge_ligue1(season1,season2):
    merged_F =open("./Data_final/data_ligue1.csv","a")    
    merged_F.write('Season,Journee,HomeTeam,AwayTeam,FTHG,FTAG,URL\n')  
    for num in range(season1,season2+1):
        ff = open("season_"+str(season1)+"_"+str(season1+1)+".csv")
        next(ff) # skip the header
        for line in ff:
            merged_F.write(f'{num}-{num+1},'+line)
        ff.close() # not really needed
    merged_F.close()

def scrap_to_list(tree,xpth,scrape_type):
    """scrape type: 1 .text, 2 .attrib["alt"]"""
    scrape_list = tree.xpath(xpth)
    list_out = []
    if scrape_type == 1:
        for each in scrape_list:
            list_out.append(each.text.strip())
    if scrape_type == 2:
        for each in scrape_list:
            list_out.append(each.attrib["alt"])        
    return list_out

# Authentification pour utiliser API twitter
def auth_cpt_twtr(consumer_key,consumer_secret,access_token,access_secret):
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    api = tweepy.API(auth)
    try:
        api.verify_credentials()
        print('Successful Authentication')
        return api
    except:
        print('Failed authentication')
        exit()

# Recuperer les tweets par un requette (query)
def get_tweets(query, api, bucket_url, count = 200):
    q=str(query)
    fetched_tweets = api.user_timeline(screen_name=q, count = count)
    clmns = ['Tweet', 'Date de creation', 'hashtags']
    data_tweets=[]
    # analyser tweets une par une
    for tweet in fetched_tweets:
        # Ajouter les tweets à la dataframe
        oneline_text = str.join(" ", tweet.text.splitlines())
        data_tweets.append([oneline_text, tweet.created_at, tweet.entities["hashtags"]])
    df = pd.DataFrame(data_tweets, columns=clmns)
    df.to_csv(r'{}{}.csv'.format(bucket_url,query), index=False)
    
def images_downloader(media_files, directory):
    path_downloaded= set()
    for media_file in media_files:
        path_downloaded.add(wget.download(media_file, directory))
    return path_downloaded

def get_photo_user(at_user,api):
    # Definir un class à partir d'une function qui
    @classmethod
    def parse(cls, api, raw):
        status = cls.first_parse(api, raw)
        setattr(status, 'json', json.dumps(raw))
        return status

    # You need to do it for all the models you need
    # Status() is the data model for a tweet
    tweepy.models.Status.first_parse = tweepy.models.Status.parse
    tweepy.models.Status.parse = parse
    # User() is the data model for a user profil
    tweepy.models.User.first_parse = tweepy.models.User.parse
    tweepy.models.User.parse = parse
    
    # This code stores all the tweets by a specific user in the variable tweets. 
    # Now, we are ready to filter those with images. In our case, we want 
    # 200 tweets which are directly created by the user (i.e., No retweets nor replies)
    tweets = api.user_timeline(screen_name=at_user,
    count=1, include_rts=False,
    exclude_replies=True)
    last_id = tweets[-1].id
    while (True):
        more_tweets = api.user_timeline(screen_name=at_user,
        since_id="2022-06-01",
        count=1, include_rts=False,
        exclude_replies=True,
        max_id=last_id-1)
        # There are no more tweets
        if (len(more_tweets) == 0):
            break
        else:
            last_id = more_tweets[-1].id-1
            tweets = tweets + more_tweets

    # Trouver les liens pour toutes les images dans chaque tweet
    media_files = set()
    for status in tweets:
        media = status.entities.get('media', [])
        if(len(media) > 0):
            media_files.add(media[0]['media_url'])
    return media_files

## parametres des années à scrapper


In [ ]:
yr_dbt = 16
currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
yr_fin = int(date.strftime("%y"))


## creation d'une destination local
1. verifier si le dossier existe sinon le creer temporairement
2. telecharger les fichiers data en question dans le dossier

In [ ]:
s_dir="./data_F1/"
if not os.path.exists(s_dir):
    os.mkdir(s_dir)
for i in range(yr_dbt,yr_fin):
    urllib.request.urlretrieve(f"https://www.football-data.co.uk/mmz4281/{i}{i+1}/F1.csv", f"./{s_dir}F1_{i}_{i+1}.csv")
#Div	Date	Time	HomeTeam	AwayTeam	FTHG	FTAG	FTR	HTHG	HTAG	HTR	HS	AS	HST	AST	HF	AF	HC	AC	HY	AY	HR	AR	B365H	B365D	B365A	BWH	BWD	BWA	IWH	IWD	IWA	PSH	PSD	PSA	WHH	WHD	WHA	VCH	VCD	VCA	MaxH	MaxD	MaxA	AvgH	AvgD	AvgA	B365>2.5	B365<2.5	P>2.5	P<2.5	Max>2.5	Max<2.5	Avg>2.5	Avg<2.5	AHh	B365AHH	B365AHA	PAHH	PAHA	MaxAHH	MaxAHA	AvgAHH	AvgAHA	B365CH	B365CD	B365CA	BWCH	BWCD	BWCA	IWCH	IWCD	IWCA	PSCH	PSCD	PSCA	WHCH	WHCD	WHCA	VCCH	VCCD	VCCA	MaxCH	MaxCD	MaxCA	AvgCH	AvgCD	AvgCA	B365C>2.5	B365C<2.5	PC>2.5	PC<2.5	MaxC>2.5	MaxC<2.5	AvgC>2.5	AvgC<2.5	AHCh	B365CAHH	B365CAHA	PCAHH	PCAHA	MaxCAHH	MaxCAHA	AvgCAHH	AvgCAHA

## creation un seul fichier data pour toutes les données scrappées

In [ ]:
merged =open(s_dir+"F1.csv","w")    
#merged.write('HomeTeam,AwayTeam,FTHG,FTAG\n')  
for yr in range(16,22):
    if yr<19 :
        f = open(f"{s_dir}F1_{yr}_{yr+1}.csv")
        freading = csv.reader(f,delimiter=',')
        next(freading) # skip the header
        for ligne in freading:
            merged.write(ligne[2]+','+ligne[3]+','+ligne[4]+','+ligne[5]+'\n')
        f.close() # not really needed
    else:
        f = open(f"{s_dir}F1_{yr}_{yr+1}.csv")
        freading = csv.reader(f,delimiter=',')
        next(freading) # skip the header
        for ligne in freading:
            merged.write(ligne[3]+','+ligne[4]+','+ligne[5]+','+ligne[6]+'\n')
        f.close() # not really needed

merged.close()

## Deplacer le BDD final (le fichier F1 avec les fichiers telechargés) à notre bucket dans le chemin destiné

In [ ]:
path_local = f"{s_dir}F1.csv"
path_bucket = "gs://bucket-git-m2i/DB/historique"
subprocess.call(f"gsutil cp {path_local} {path_bucket}", shell=True)
for i in range(yr_dbt,yr_fin):
    subprocess.call(f"gsutil cp ./{s_dir}F1_{i}_{i+1}.csv {path_bucket}", shell=True)


## Liberer l'espace local par supprimer le dossier creer temporairement

In [ ]:
!rm -r $s_dir

## Scrap du calendrier des matchs à venir -> construction d'un fichier CSV par journée qui liste les 10 matchs (home, away, url)

In [3]:
# journée à scrapper
for journee in range(1,39): # journée de 1 à 23 (index final = 24-1)
    # url de la site à scrapper
    url_a_scrapper = f"https://www.ligue1.fr/calendrier-resultats?seasonId=2022-2023&matchDay={journee}"
    # url principale de la site
    url_base = "https://www.ligue1.fr"
    # request pour avoir le corps de code de la page à scrapper
    resp = requests.get(url_a_scrapper)
    # trier la contnue du code de la page comme un arbre de html
    tree = html.fromstring(resp.content)

    # scrape toutes les id des matchs dans la page
    match_id = []
    match_id_code = tree.xpath('/html/body/main/div[3]/div[2]/div/div[2]/ul[*]/li/a/div[2]')
    for each_id in match_id_code:
        # if va chercher dans chaque balise a id son contenu 
        # et l'ajouter dans la liste de match_id
        match_id.append(each_id.attrib["id"])

    # recuperer les elements à chercher par leurs xpath
    equipe_home = []
    equipe_away = []
    match_href = []
    match_url = []
    i=0
    for each_id in match_id:
        # le resultat de tree.xpath sera en forme de liste 
        # il faut index[0] et .text pour recuperer le contenu
        equipe_home.append(nom_normalizer(tree.xpath(f'//*[@id="{each_id}_match-result"]/a/div[1]/div/span[1]')[0].text))
        equipe_away.append(nom_normalizer(tree.xpath(f'//*[@id="{each_id}_match-result"]/a/div[3]/div/span[1]')[0].text))
        # Afin d'avoir acceder à la page de detailles de match
        # on scrape le lien de chaque match et on l'ajout à la url_base
        # afin de deviner un lien complet
        # ici le resultat de tree.xpath a plusieurs attributes
        # donc on choisis l'attribue concerné par [0].attrib["href"]
        # href c'est l'attribue nous interesse
        match_href.append(tree.xpath(f'//*[@id="{each_id}_match-result"]/a'))
        match_url.append(url_base + match_href[i][0].attrib["href"])
        i+=1
#     for i in range(10):
#         print(i+1," - ",equipe_home[i],"\t", equipe_away[i],"\n url: ", match_url[i],"\n")

    # zipper les informations des listes dans un dataframe
    # avec 3 columns correspondant à chaque liste
    # et les sauvegarder dans un fichier CSV
    df_journee_1 = pd.DataFrame(list(zip(equipe_home, equipe_away, match_url)), 
                        columns =['Equipe HOME', 'Equipe AWAY', 'URL']) 
    df_journee_1.to_csv(f'gs://bucket-git-m2i/DB/calendrier/journee_{journee}.csv', index=False)

## Merging des csv issus du scrap calendrier -> construction d'un csv qui comprend la journée, home, away et l'url

In [4]:
merged =file_io.FileIO("gs://bucket-git-m2i/DB/calendrier/merging.csv",mode="w")    
merged.write('Journée,Equipe_Home,Equipe_Away,URL\n')  
for num in range(1,39):
    f = file_io.FileIO("gs://bucket-git-m2i/DB/calendrier/journee_"+str(num)+".csv",mode="r")
    next(f) # skip the header
    for line in f:
        merged.write(f'{num},'+str(line))
    f.close() # not really needed
merged.close()

## Merging des csv contenant les résultats historiques des 5 dernières saisons -> csv comprenant home, away, score_home, score_away

In [5]:
merged_F =file_io.FileIO("gs://bucket-git-m2i/DB/historique/merging_F.csv",mode="w")    
merged_F.write('HomeTeam;AwayTeam;FTHG;FTAG\n')  
for num in range(1,3):
    ff = file_io.FileIO("gs://bucket-git-m2i/DB/historique/F"+str(num)+".csv",mode="r")
    next(ff) # skip the header
    for line in ff:
        merged_F.write(line)
    ff.close() # not really needed
merged_F.close()

## Se basant sur merging historique -> calcul (dans variables) des statistiques de la ligue (toutes équipes confondues)

In [6]:
df = pd.read_csv('gs://bucket-git-m2i/DB/historique/merging_F.csv', sep=';')
### normalizer tous les noms des equipes dans le dataframe
# Home 
df['HomeTeam'] = df['HomeTeam'].str.strip().str.upper()
df['HomeTeam'] = df['HomeTeam'].apply(nom_normalizer)
# Away
df['AwayTeam'] = df['AwayTeam'].str.strip().str.upper()
df['AwayTeam'] = df['AwayTeam'].apply(nom_normalizer)

# print(df)

home_total = df[['FTHG']].sum()[0]
home_average = df[['FTHG']].mean(numeric_only=True)[0]
nb_of_matches = df[['HomeTeam']].count()[0]
# print(home_total)
# print(home_average)
# print(nb_of_matches)

away_total = df[['FTAG']].sum()[0]
away_average = df[['FTAG']].mean(numeric_only=True)[0]
# print(away_total)
# print(away_average)

## Calcul d'un scoring de chaque équipe (indépendemment de son adversaire) selon away/home relatif aux buts encaissés et marqués -> création csv reprenant ces résultats

In [7]:
with file_io.FileIO('gs://bucket-git-m2i/DB/calendrier/merging.csv',mode="r") as calendrier:
    csv_reader = csv.reader(calendrier)
    with file_io.FileIO('gs://bucket-git-m2i/DB/golden/final-results.csv', mode='w') as new_file:
        # on ouvre un csv pour sauvegarder les resultats
        csv_writer = csv.writer(new_file)                

        for line in calendrier:# line est un chaine des caracs
            data_line = line.split(sep=',') # separer les donnée
            domicile = nom_normalizer(data_line[1].strip().upper()) # recuperer l'equipe domicile
            visiteur = nom_normalizer(data_line[2].strip().upper()) # recuperer l'equipe visiteur
            
            
            team_home_df = df[df['HomeTeam']==domicile]
            team_home_total = team_home_df[['FTHG']].sum()[0]
            team_home_average = team_home_df[['FTHG']].mean()[0]
            home_conceed_total = team_home_df[['FTAG']].sum()[0]
            home_conceed_average = team_home_df[['FTAG']].mean()[0]
            nb_matches_home = team_home_df[['FTAG']].count()[0]
            home_attack_str = (team_home_average / home_average)
            home_defence_str = (home_conceed_average / away_average)

            team_away_df = df[df['AwayTeam']==visiteur]
            away_score_total = team_away_df[['FTAG']].sum()[0]
            away_score_average = team_away_df[['FTAG']].mean()[0]
            away_conceed_total = team_away_df[['FTHG']].sum()[0]
            away_conceed_average = team_away_df[['FTHG']].mean()[0]
            nb_matches_away = team_away_df[['FTHG']].count()[0]
            away_attack_str = (away_score_average / away_average)
            away_defence_str = (away_conceed_average / home_average)

            #csv_writer.writerow(str(away_attack_str)+','+str(away_defence_str)+'\n')
            new_file.write(str(domicile)+','+str(visiteur)+','+str(home_attack_str).strip()+','+str(home_defence_str).strip()+','+str(away_attack_str).strip()+','+str(away_defence_str).strip()+'\n')

## Calcul des probabilités du résultat (draw / home_win / away_win) -> création csv reprenant ces résultats

In [8]:
with file_io.FileIO('gs://bucket-git-m2i/DB/golden/final-results.csv', mode='r') as matchs:
    csv_reader = csv.reader(matchs)
    with file_io.FileIO('gs://bucket-git-m2i/DB/golden/match-prob.csv', mode='w') as probs:
        # on ouvre un csv pour sauvegarder les resultats
        csv_writer = csv.writer(probs) 
        for line in matchs: # line est un chaine des caracs
            data_line = line.split(sep=',') # separer les donnée
            domicile = nom_normalizer(data_line[0].strip().upper()) # recuperer l'equipe domicile
            visiteur = nom_normalizer(data_line[1].strip().upper()) # recuperer l'equipe visiteur
            away_attack_str = data_line[2].strip()
            away_defence_str = data_line[3].strip()
            home_defence_str = data_line[4].strip()
            home_attack_str = data_line[5].strip()

            home_expect = float(home_attack_str) * float(away_defence_str) * float(home_average)
            away_expect = float(away_attack_str) * float(home_defence_str) * float(away_average)
            def poisson_probability(l, x):
                probability = ((l**x) * math.exp(-l)) / math.factorial(x)
                return probability*100
            home_goals_prob = []
            for i in range(8):
                expect = poisson_probability(home_expect, i)
                home_goals_prob.append(expect)
            home_goals_probs = np.round(home_goals_prob,2)
            away_goals_prob = []
            for i in range(8):
                expect = poisson_probability(away_expect, i)
                away_goals_prob.append(expect)  
            away_goals_probs = np.round(away_goals_prob,2) 
            p = {'Home0':[((home_goals_probs[0]*away_goals_probs[0])/100), ((home_goals_probs[0]*away_goals_probs[1])/100), ((home_goals_probs[0]*away_goals_probs[2])/100), ((home_goals_probs[0]*away_goals_probs[3])/100), ((home_goals_probs[0]*away_goals_probs[4])/100), ((home_goals_probs[0]*away_goals_probs[5])/100),((home_goals_probs[0]*away_goals_probs[6])/100),((home_goals_probs[0]*away_goals_probs[7])/100)], 
                'Home1':[((home_goals_probs[1]*away_goals_probs[0])/100), ((home_goals_probs[1]*away_goals_probs[1])/100), ((home_goals_probs[1]*away_goals_probs[2])/100), ((home_goals_probs[1]*away_goals_probs[3])/100), ((home_goals_probs[1]*away_goals_probs[4])/100), ((home_goals_probs[1]*away_goals_probs[5])/100),((home_goals_probs[1]*away_goals_probs[6])/100),((home_goals_probs[1]*away_goals_probs[7])/100)], 
                'Home2':[((home_goals_probs[2]*away_goals_probs[0])/100), ((home_goals_probs[2]*away_goals_probs[1])/100), ((home_goals_probs[2]*away_goals_probs[2])/100), ((home_goals_probs[2]*away_goals_probs[3])/100), ((home_goals_probs[2]*away_goals_probs[4])/100), ((home_goals_probs[2]*away_goals_probs[5])/100),((home_goals_probs[2]*away_goals_probs[6])/100),((home_goals_probs[2]*away_goals_probs[7])/100)], 
                'Home3':[((home_goals_probs[3]*away_goals_probs[0])/100), ((home_goals_probs[3]*away_goals_probs[1])/100), ((home_goals_probs[3]*away_goals_probs[2])/100), ((home_goals_probs[3]*away_goals_probs[3])/100), ((home_goals_probs[3]*away_goals_probs[4])/100), ((home_goals_probs[3]*away_goals_probs[5])/100),((home_goals_probs[3]*away_goals_probs[6])/100),((home_goals_probs[3]*away_goals_probs[7])/100)], 
                'Home4':[((home_goals_probs[4]*away_goals_probs[0])/100), ((home_goals_probs[4]*away_goals_probs[1])/100), ((home_goals_probs[4]*away_goals_probs[2])/100), ((home_goals_probs[4]*away_goals_probs[3])/100), ((home_goals_probs[4]*away_goals_probs[4])/100), ((home_goals_probs[4]*away_goals_probs[5])/100),((home_goals_probs[4]*away_goals_probs[6])/100),((home_goals_probs[4]*away_goals_probs[7])/100)], 
                'Home5':[((home_goals_probs[5]*away_goals_probs[0])/100), ((home_goals_probs[5]*away_goals_probs[1])/100), ((home_goals_probs[5]*away_goals_probs[2])/100), ((home_goals_probs[5]*away_goals_probs[3])/100), ((home_goals_probs[5]*away_goals_probs[4])/100), ((home_goals_probs[5]*away_goals_probs[5])/100),((home_goals_probs[5]*away_goals_probs[6])/100),((home_goals_probs[5]*away_goals_probs[7])/100)],
                'Home6':[((home_goals_probs[6]*away_goals_probs[0])/100), ((home_goals_probs[6]*away_goals_probs[1])/100), ((home_goals_probs[6]*away_goals_probs[2])/100), ((home_goals_probs[6]*away_goals_probs[3])/100), ((home_goals_probs[6]*away_goals_probs[4])/100), ((home_goals_probs[6]*away_goals_probs[5])/100),((home_goals_probs[6]*away_goals_probs[6])/100),((home_goals_probs[6]*away_goals_probs[7])/100)],
                'Home7':[((home_goals_probs[7]*away_goals_probs[0])/100), ((home_goals_probs[7]*away_goals_probs[1])/100), ((home_goals_probs[7]*away_goals_probs[2])/100), ((home_goals_probs[7]*away_goals_probs[3])/100), ((home_goals_probs[7]*away_goals_probs[4])/100), ((home_goals_probs[7]*away_goals_probs[5])/100),((home_goals_probs[7]*away_goals_probs[6])/100),((home_goals_probs[7]*away_goals_probs[7])/100)]}
            probability = pd.DataFrame(p, index=['away0','away1','away2', 'away3', 'away4', 'away5', 'away6', 'away7'])
            nump = probability.to_numpy()
            draw_prob = np.trace(nump)
            Home_win_prob = (np.trace(nump, offset = 1))+(np.trace(nump, offset = 2))+(np.trace(nump, offset = 3))+(np.trace(nump, offset = 4))+(np.trace(nump, offset = 5))+(np.trace(nump, offset = 6))
            away_win_prob = (np.trace(nump, offset = -1))+(np.trace(nump, offset = -2))+(np.trace(nump, offset = -3))+(np.trace(nump, offset = -4))+(np.trace(nump, offset = -5))+(np.trace(nump, offset = -6))
            probs.write(domicile+','+visiteur+','+str(draw_prob).strip()+','+str(Home_win_prob)+','+str(away_win_prob)+','+'\n')
matchs.close()
probs.close()